In [ ]:
import numpy as np
from matplotlib import pyplot

In [ ]:
def f(x):
    return np.power(2., x)-1.

In [ ]:
x = np.linspace(0., 1., 100, dtype=np.float32)
y = f(x)
poly = np.polyfit(x, y, deg=2).astype(np.float32)
fit = np.polyval(poly, x.astype(np.float32))

pyplot.plot(x, y)
pyplot.plot(x, fit, color='r')

In [ ]:
poly

In [ ]:
np.finfo(np.float32)

In [ ]:
x = np.float32(0.)
print (bin(x.view(dtype=np.uint32).copy()))
exponent = ((x.view(dtype=np.uint32).copy() >> 23) & (255)) - 127
exponent

In [ ]:
np.log2(np.e)

In [ ]:
def exp_approx(x):
        # http://spfrnd.de/posts/2018-03-10-fast-exponential.html
        x = x * np.log2(np.e)
        xi = np.int32(np.floor(x))
        xf = x - xi
        y = np.power(np.float32(2.), xi)
        corr = np.float32(1.) + np.polyval(poly, xf).astype(np.float32)
        iview = corr.view(dtype=np.uint32)
        print (bin(iview[0]))
        iview &= np.uint32(~(255 << 23))
        iview |= (((xi + 127) << 23)).astype(np.uint32)
        print (bin(iview[0]))
        return corr
        #return y*corr

In [ ]:
bin(np.uint32(~(255 << 23)))

In [ ]:
logs = np.linspace(7., 10., 10000)
y = exp_approx(logs)
ref = np.exp(logs)

In [ ]:
rel_err = np.abs(ref - y) / ref
print (rel_err.max())
pyplot.plot(logs, rel_err)
pyplot.gca().set(yscale='log')

In [ ]:
pyplot.plot(logs, y)
pyplot.plot(logs, ref)
pyplot.gca().set(yscale='log')

In [ ]:
from path_guiding import ExpApproximation

logs = np.linspace(-30., 10., 1024, dtype=np.float32)
ref = np.exp(logs)

# Vectorized version
M = 8
y = np.zeros(len(logs), dtype=np.float32)
for i in range(0, 1024, M):
    y[i:i+M] = ExpApproximation(logs[i:i+M])


pyplot.plot(logs, np.abs(ref - y) / ref)
pyplot.gca().set(yscale='log')
pyplot.show()

pyplot.plot(logs, y)
pyplot.plot(logs, ref)
pyplot.gca().set(yscale='log')
pyplot.show()

del y

# Scalar version
M = 8
y = np.array([ ExpApproximation(l) for l in logs ], dtype=np.float32)
pyplot.plot(logs, np.abs(ref - y) / ref)
pyplot.gca().set(yscale='log')
pyplot.show()

Fast $\log_2$ approximation
--------------------------------------

Based on the floating point representation $x=m2^e$. Hence
\begin{equation}
\log_2 x = \log_2 m + e
\end{equation}
where $m \in [1,2)$.

In [ ]:
# Need a fit for values in [1,2]
x = np.linspace(0.9, 2.1, 100, dtype=np.float32)
y = np.log2(x)
log2_poly = np.polyfit(x, y, deg=3).astype(np.float32)
fit = np.polyval(log2_poly, x.astype(np.float32))

pyplot.plot(x, y)
pyplot.plot(x, fit, color='r')

In [ ]:
def log2_taylor(x):
    # Expansion around x=1
    x = x-np.float32(1)
    y = x*(np.float32(1.) + x*(np.float32(1./3.)*x - np.float32(1./2.)))
    y /= np.float32(np.log(2.))
    return y

def log2_approx(x):
    iview = x.view(dtype=np.uint32)
    #print ('bin view = {}'.format(bin(iview)))
    exponent = iview & np.uint32(255 << 23)
    exponent = (exponent >> 23) - 127
    mantisse = (iview & np.uint32(~(255 << 23)) | np.uint32((127 << 23)))
    mantisse = mantisse.view(dtype=np.float32)
    # The mantisse always lies between 1 and 2
    # Close to x=1, the general log2_poly-based approximation diverges from the true value.
    # So these cases are treated separately.
    if exponent == -1 and mantisse>1.6:
        # The argument lies close to 1 due to division by 2.
        return log2_taylor(mantisse/np.float32(2))
    elif exponent == 0 and mantisse<1.2:
        return log2_taylor(mantisse)
    else:
        return np.polyval(log2_poly, mantisse) + exponent

In [ ]:
def f(x):
    return log2_approx(np.float32(x))

In [ ]:
x = 2.**(-0.0000001)
y = f(x)
yref = np.log2(np.float32(x))
print(x, y, yref, (y-yref)/yref)

In [ ]:
np.log2(1.9999998807907104) - 1.

In [ ]:
np.log2(1.9999998807907104 * 0.5)

In [ ]:
logs = np.linspace(-2., 2., 10000, dtype=np.float32)
x = np.power(np.float32(2.),logs)

yref = np.log2(x)
y = np.array([f(x_) for x_ in x ])

rel_err = np.abs(yref - y) / np.abs(yref)
pyplot.plot(logs, rel_err)
pyplot.gca().set(yscale='log')